### LAB01 - Introduction to Semantic Kernel

In this lab, we will learn how to use Semantic Kernel to interact with Large Language Models. We'll complete the following steps:

1) Import general dependencies.
2) Using Semantic Kernel SDK to run prompts on a Text Completion model from Azure OpenAI. 
3) Instantiate a Kernel to add LLM capabilities to your application.
4) Connect the Kernel to a Completion model to run your first semantic function.

#### Step 1 - Import Dependencies

**Semantic Kernel** is a SDK currently available in C#, Java and Python. However, each language support different at the current stage. Please visit [supported languages](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages) to check which features are supported per language.

In this lab, we'll use Python SDK which can be obtained in Pypi repository.

**Important:** if you get an error running the code below, make sure that you've completed the setup process. Optionally, you can run `pip install semantic_kernel` in your terminal or `!python -m pip install semantic_kernel` in a Python code cell.

In [ ]:
#Importing the Semantic Kernel SDK that will be used in this lab. 
#The other dependencies will be imported directly in the step where they are used.
import semantic_kernel as sk

#### Step 2 - Using Semantic Kernel SDK to run prompts on a Text Completion model from Azure OpenAI

In this lab, we'll connect to an **AI Service** and use a Text Completion model from **Azure OpenAI Service**. 
The Semantic Kernel also supports **OpenAI**. Other AI Services, like **Hugging Face** will be supported shortly. You can visit this [link](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages#ai-service-endpoints) to check the most updated list.

In this example, we're using **AzureTextCompletion** module to interact with Text Completion models from Azure OpenAI, such as text-davinci-003. This is suitable for text generation activities. We are also using a module to control the behavior of the requests, defining parameters like Temperature, max_tokens and even how many responses we want to generate.   

**Important:** We're a using a method to automatically read sensitive parameters from a `.env` file. Therefore, please ensure that you have a `.env` file in the same directory of this notebook with the following parameters:

``` python
AZURE_OPENAI_DEPLOYMENT_NAME="NAME OF YOUR MODEL DEPLOYMENT"
AZURE_OPENAI_ENDPOINT="https://<YOUR_ENDPOINT>.openai.azure.com/"
AZURE_OPENAI_API_KEY="KEY"
```
Optionally, you can update the values and run the cell below to create your `.env` file.

In [6]:
%%writefile .env
AZURE_OPENAI_DEPLOYMENT_NAME="NAME OF YOUR MODEL DEPLOYMENT"
AZURE_OPENAI_ENDPOINT="https://<YOUR_ENDPOINT>.openai.azure.com/"
AZURE_OPENAI_API_KEY="KEY"

Overwriting .env


In [ ]:
#Importing the Azure Text Completion service connector
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion
#Importing the module that contains the settings for the Azure Text Completion service
from semantic_kernel.connectors.ai import CompleteRequestSettings

# Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

# Create the Azure Text Completion service connector
azure_text_completion = AzureTextCompletion(deployment, endpoint, api_key)

# Create the settings to control the behavior to the requests of Azure Text Completion service
request_settings = CompleteRequestSettings(
    max_tokens=20,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=2
)

Now that we have all set, let's run a simple prompt to generate taglines using the Text Completion model. Please note that the number of results, response lenght and temperature are controlled by the **CompleteRequestSettings**.

In [ ]:
prompt = "Write a tagline for a Telecom company."

#Calling the Azure Text Completion service using Semantic Kernel SDK and AI Service Connector
results = await azure_text_completion.complete_async(prompt, request_settings)

#Printing out the results
i = 1
for result in results:
    print(f"Result {i}: {result}")
    i += 1

#### Step 3 - Instantiate a Kernel to add LLM capabilities to your application
The core of Semantic Kernel is it's Kernel. It's responsible to transform an User Input (or ASK) into a response by orchestrating Plugins. It's also what makes Semantic Kernel shines as it can transform existing applications in AI applications powered by LLMs. 

In this step, we'll instantiate a simple Kernel with no additonal options. Optionally, we could define Logging, Memory and the set of skills we want to use. 

In [ ]:
#Instantiating the kernel
kernel = sk.Kernel()


#Example of how to instantiate the kernel with a different log level
#logger = sk.NullLogger()
#kernel_with_logging = sk.Kernel(log=logger)

#### Connect the Kernel to a Completion model to run your first semantic function.
Now that we have our kernel, we can start adding capabilities to it. We already saw how to use the SDK and connectors to the AI services to run prompts. Now, let's see how can we use the Kernel to transform a prompt into a semantic function able to run Text Completion activities using a LLM.

The first step is to add to our kernel the Text Completion model that we already have defined before. We can add multiple models and give then names (or nicknames) to later decide which ones we want to use and define programmatically which one is the default.

You can also add Chat Completion models, like GPT-4 and GPT-3.5-Turbo, as per the commented code.

In [ ]:
# Add the text completion service to the kernel
kernel.add_text_completion_service("Text Completion - Text-DaVinci-003", 
                                   AzureTextCompletion(deployment, endpoint, api_key))

#We can add several models of the same type to the kernel
#kernel.add_text_completion_service("Other Text Completion",
#                                   AzureTextCompletion("text-curie-001", endpoint, api_key))

#We can add other models types to the kernel, like ChatCompletions (GPT-35-Turbo, GPT4)
#from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
#kernel.add_chat_completion_service("ChatCompletion - GPT-4",
#                                   AzureChatCompletionCompletion("text-curie-001", endpoint, api_key))

#kernel.set_default_text_completion_service("Text Completion - Text-DaVinci-003")

With the kernel connected to a large language model from Azure OpenAI Service, we can start using it to run Text Completion operations.

We'll define a **Semantic Function** in order to run prompts using a LLM. We'll deep-dive on Semantic Functions later, so let's just focus on the basics in this step of the lab.

You can add a **Semantic Function** to the Kernel based on a **Prompt**. This is simple a textual description, or instruction, of what we want the function to do. Optionally, we can add parameters, like `{{$input}}`, to inject values in runtime.

In this example, we'll create a function to classify a Telecom problem in different categories, like Mobile Internet, Fixed Internet, etc. As **input**, we'll use the problem description. Please note that as we define the function we can also control the behavior of how we expect the kernel to run it, for example, controlling the max number of tokens in the response and the temperature. This we'll ensure that the function we'll run consistenly.

In [ ]:
# Set the prompt for classifying the problem description
prompt = """Problem Description: {{$problem}}
Classify the problem description above as one of these categories: Fixed Internet, Mobile Internet, TV, Landline, Billing.
Only write the output category with no extra text.
Only write one category per problem description.
"""

# Create a semantic function for classifying the problem description
classify_call_function = kernel.create_semantic_function(prompt, 
                                                         max_tokens=10,
                                                         temperature=0.5)

Now that our Kernel has a new function to classify problems, we'll test it based on a problem description. We expected as a result one of the classifications previously defined in the prompt.

In [ ]:
problem = "My 5G connection is terrible at my bedroom."


# Call the classify function with a problem description
classification = classify_call_function(problem)

# Print the result
print(f"Result: {classification}")